# Load Data + SMOTE

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

df = pd.read_csv("../data/raw/creditcard.csv")

X = df.drop("Class", axis=1)
y = df["Class"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

smote = SMOTE(random_state=42)
X_train_sm, y_train_sm = smote.fit_resample(X_train, y_train)


/Users/waheedkehinde/Desktop/fraud-detection-app/venv/lib/python3.9/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


# Define Parameter Grid (KEEPING IT SMALL)

In [12]:
from sklearn.ensemble import RandomForestClassifier

# Fast, high-performing RF
best_rf = RandomForestClassifier(
    n_estimators=200,     # enough trees for stability
    max_depth=15,         # limits overfitting
    min_samples_leaf=2,   # regularization
    random_state=42,
    n_jobs=-1
)

best_rf.fit(X_train_sm, y_train_sm)


RandomForestClassifier(max_depth=15, min_samples_leaf=2, n_estimators=200,
                       n_jobs=-1, random_state=42)

# GridSearch with Cross-Validation

# Best Parameters

# Evaluating Tuned Model

In [13]:
from sklearn.metrics import classification_report, roc_auc_score

# Probabilities
y_probs_best = best_rf.predict_proba(X_test)[:, 1]
roc_auc_best = roc_auc_score(y_test, y_probs_best)
print("ROC-AUC:", roc_auc_best)

# Predictions
y_pred_best = best_rf.predict(X_test)
print(classification_report(y_test, y_pred_best))


ROC-AUC: 0.9810598937098757
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.69      0.86      0.76        98

    accuracy                           1.00     56962
   macro avg       0.84      0.93      0.88     56962
weighted avg       1.00      1.00      1.00     56962



##  Summary: Fast RF Hyperparameter Tuning

A Random Forest was trained directly on SMOTE-balanced data with strong default
hyperparameters (n_estimators=200, max_depth=15, min_samples_leaf=2)
to reduce training time on a laptop.

The model achieves:
- ROC-AUC ≈ 0.981
- Precision ≈ 69%
- Recall ≈ 86%
- F1 ≈ 0.76 for fraud detection

This model provides a robust baseline for further threshold tuning
and deployment considerations.
